In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [2]:
%matplotlib nbagg

In [3]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from matplotlib import cm
from libertem import api

In [4]:
ctx = api.Context()

In [5]:
ds = ctx.load(
    "raw",
    path="C:/Users/weber/ownCloud/Projects/Open Pixelated STEM framework/Data/EMPAD/scan_11_x256_y256.raw",
    dtype="float32",
    scan_size=(256, 256),
    detector_size_raw=(130, 128),
    crop_detector_to=(128, 128),
)
(scan_y, scan_x, detector_y, detector_x) = ds.shape
mask_shape = np.array((detector_y, detector_x))
cx = detector_x/2
cy = detector_y/2

In [6]:
def sparse_one():
    return sp.csr_matrix(([1], ([int(cy)], [int(cx)])), shape=(detector_y, detector_x))
print(sparse_one())

  (64, 64)	1


In [7]:
sparse_job = ctx.create_mask_job(dataset=ds, factories=[sparse_one])

In [8]:
%time sparse_result = ctx.run(sparse_job)

NotImplementedError: Sparse matrices not supported yet by back-end